# **Preliminary:** Encoding peptide sequences

## Intro

In their digital form, DNA and Proteins are usually made up of sequences of letters -- nucleic acids (AATTGCA...) for the former and amino acids for the latter (MNPLLILTFV...) -- and while this form is relatively understandable for humans, it is difficult to work with for machine learning problems. It is important that we convert these sequences into something more friendly for learning algorithms, and that we think carefully about how will we do the conversion, since the algorithms available to us will depend highly on this step. To start out, I will list a few different options for a single peptide sequence, say PEPTIDEK.

These are non-exhaustive but they should be useful in a broad range of circumstances. At first, I am going to talk largely abstractly about each of the different ways for encoding, and we will push off talking about their numpy representations until later.

#### **Amino Acid Counts**

One solution, is to list all possible amino acids and count their occurences in each of the peptides you are working with. For PEPTIDEK, this would look like:

```
 #A #C #D #E #F #G #H #I #K #L #M #N #P #Q #R #S #T #V #W #Y
  0  0  1  2  0  0  0  1  1  0  0  0  2  0  0  0  1  0  0  0
```

This a perfectly legitimate way to encode sequences, and has been popular for a long time. If your hypothesis is that what you are trying to predict, say retention time, is entirely determined by the composition of a peptide, then this is fine. Values for individual amino acids, such as hydrophobicity, can be combined with your counts to create a single number to represent each peptide.

What this loses, however, is the postional information intrinsic to each amino acid. Do basic amino acids behave differently when postioned next to acidic amino acids? How does a proline behave at different positions in an amino acid sequence? You wouldn't be able to recover this information.

#### **One Hot Encoding**

One of the most popular ways to retain position information is to use a one hot encoding of the sequence. This techinque envisions the sequence as a series of columns for each possible amino acid, with a series of rows representing a single postion in the sequence. We put a 1 where an amino acid appears and a 0, where it doesn't. Again, for the sequence PEPTIDEK, this would look like,

```
    A C D E F G H I K L M N P Q R S T V W Y
   ----------------------------------------
 0| 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1| 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2| 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 3| 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 4| 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 5| 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 6| 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 7| 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
```

If you have not come accross this before, I encourage you to go through the above matrix and make sure that it does in fact spell PEPTIDEK. A representation like this contains a great deal of information. It encodes the amino acids and there position throughout the entire sequence. A good algorithm will make use of both, attempting to learn the effect of individual amino acid types on the property we want to predict as well as the sequence context those amino acids occur in.

It should be noted that you can arrive back at the Amino acid counts representation by walking down each individual column and counting up the number of 1's that you find.

#### **Integer encoding**

An encoding closely related to the one above is the integer encoding. If we were to take a one hot encoding and walk accross the rows, recording each time which column a 1 appears in, we get a sequence of integers representing our sequence of amino acids. So for the first amino acid, we look at our sequence and see a P, which in the matrix would be column 13. The reason I start numbering the columns at 1 is due to padding, which will be explained bellow. For the sequence PEPTIDEK, this would look like,

```
[13, 4, 13, 17, 8, 3, 4, 9]
```

This representation provides much of the same information as one hot encoding, but makes unlocking certain functionality in deep learning packages a bit more streamlined.

#### **Common Pitfalls**

**Modifications.** Chemical modifications are ubiquitous in genomics and proteomics. In just about every mass spec run, you will encounter oxidated methionine and phosphorylated residues that can have extremely different properties than their unmodified counterparts. These will be encoded in various ways depending on the programs in your pipeline. Here are a few examples for methionine oxidation,

- M[15.9949]
- M[16]
- M*
- M\<ox>

In general, you will need to be careful jumping between these representatioins to tell any previously trained models that these are the same. In this notebook, we will focus on just one of these cases, and I will make note of how to deal with this kind of situation near the end.

**Different length sequences.** This is of course something that you can't really avoid. Peptide sequences are almost never of uniform size, unless they are engineered to be so. Some deep learning frameworks are more forgiving about this than others, but at some point you run into a step where all the inputs need to be of uniform length. We usually do this with padding, i.e. the addition of dummy characters, usually 0, in order to extend a sequence to a final length. Here are the above examples for PEPTIDEK if we wanted it to have a final length of 10.

```
One Hot Encoded:

    A C D E F G H I K L M N P Q R S T V W Y
   ----------------------------------------
 0| 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1| 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2| 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 3| 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 4| 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 5| 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 6| 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 7| 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 8| 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 9| 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Integer Encoded:

[13, 4, 13, 17, 8, 3, 4, 9, 0, 0]
```

How to do this uniformly accross sequences will be shown below.

## Tokenizing a list of sequences

Let's take the following list of peptides from trypsin as our input sequences. The major thing to notice here is that while a few amino acids occur in all the sequences, some amino acids only occur in one. The sequences are also all of different sizes so we will get some practice with padding.

In [1]:
sequences = ["IIRHPQYDR",
             "TLNNDIM[16]LIK",
             "VSTIS[80]LPTAPPATGTK"]

#### **1. Conversion to tokens**

Since we want python to automatically register individual amino acids and their modifications, we are going to need to create a regular expression to recognize each token in turn. Here, the regular expression `"[A-Z][^A-Z]*"` will work just fine. It will first recognize a single amino acid `[A-Z]` and pick up any non-alphabetical letters that trail if present, like `[80]`.

In [2]:
import re

regex = "[A-Z][^A-Z]*"
tok_sequences = [re.findall(regex, seq) for seq in sequences]

for seq in tok_sequences:
    print(seq)

['I', 'I', 'R', 'H', 'P', 'Q', 'Y', 'D', 'R']
['T', 'L', 'N', 'N', 'D', 'I', 'M[16]', 'L', 'I', 'K']
['V', 'S', 'T', 'I', 'S[80]', 'L', 'P', 'T', 'A', 'P', 'P', 'A', 'T', 'G', 'T', 'K']


#### **2. Building our vocabulary**

Now that we have each sequence broken up into its individual amino acids, we need a way to determine all the amino acids that occur at least once. We will make use of two of pythons internal components to make our lives easier. The `set` obect will automatically deduplicate the tokens in our data, which is what we want. However, it requires them to be one after another, and not grouped into lists. To help, we can use `chain` from the `itertools` module, and specifically its `from_iterable` function, to automatically chain together the lists and feed them to the `set` constructor. I usually sort the tokens to offer some consistency between workflows.

In [3]:
from itertools import chain

aa_set = set( chain.from_iterable(tok_sequences) )
sorted_aa = sorted(aa_set)
vocab = {aa: ind for ind, aa in enumerate(sorted_aa, 1)}

print(vocab)

{'A': 1, 'D': 2, 'G': 3, 'H': 4, 'I': 5, 'K': 6, 'L': 7, 'M[16]': 8, 'N': 9, 'P': 10, 'Q': 11, 'R': 12, 'S': 13, 'S[80]': 14, 'T': 15, 'V': 16, 'Y': 17}


This should give you an automatically built vocab in the end. I would suggest manually writing an explicit vocab and saving it in a variable for projects where you want a lot of consistency, but for quick analyses, this works well. As I said above, you may need to tell your pipelines explicitly that two ways of writing oxidized methionine are actually the same. In order to do that, it may be easiest to just add a single token like so.

In [4]:
vocab["M<ox>"] = 8

print(vocab)

{'A': 1, 'D': 2, 'G': 3, 'H': 4, 'I': 5, 'K': 6, 'L': 7, 'M[16]': 8, 'N': 9, 'P': 10, 'Q': 11, 'R': 12, 'S': 13, 'S[80]': 14, 'T': 15, 'V': 16, 'Y': 17, 'M<ox>': 8}


#### **3. Encoding**

We now have all we need to make our encodings. Regardless of which method I choose, I will usually build a blank matrix using `np.zeros` of the correct size and fill it in with encodings. This matrix is left as int32 for the integer encoding but as float32 for the one hot encoding. Deep learning frameworks often expect these types when they recieve the data.

At this point, there is probably several ways to do this in python. However, I think this works just fine and is quite easy to read.

In [5]:
import numpy as np

**Integer Encoding**

In [6]:
vocab_size = max(vocab.values())
final_seq_length = max(map(len, tok_sequences))

enc_sequences = np.zeros((len(tok_sequences), 
                          final_seq_length),
                         dtype=np.int32)
for row_ind, seq in enumerate(tok_sequences):
    for col_ind, tok in enumerate(seq):
        enc_sequences[row_ind, col_ind] = vocab[tok]

print(enc_sequences)

[[ 5  5 12  4 10 11 17  2 12  0  0  0  0  0  0  0]
 [15  7  9  9  2  5  8  7  5  6  0  0  0  0  0  0]
 [16 13 15  5 14  7 10 15  1 10 10  1 15  3 15  6]]


**One Hot Encoding**

In [7]:
vocab_size = max(vocab.values())
final_seq_length = max(map(len, tok_sequences))

enc_sequences = np.zeros((len(tok_sequences), 
                          final_seq_length,
                          vocab_size),
                         dtype=np.float32)
for row_ind, seq in enumerate(tok_sequences):
    for col_ind, tok in enumerate(seq):
        enc_sequences[row_ind, col_ind, vocab[tok] - 1] = 1.

print(enc_sequences)

[[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0.

#### **4. Wrapping everything together in a function**

Alright, we have now seen how to tokenize our sequences, build a vocabulary, and convert our sequences to encodings. We can put all of these together into a function to package up the pipeline and make it easily reusable. The only required argument is the sequences, but I provided custom arguments for each step which can be used to enforce consistency accross datasets. You can confirm that this gives the same results as above.

In [8]:
import re
from itertools import chain
import numpy as np

def one_hot_encode(sequences, regex="[A-Z][^A-Z]*", vocab=None, seq_len=0):
    # 1. Conversion to tokens
    tok_sequences = [re.findall(regex, seq) for seq in sequences]

    # 2. Build vocabulary if not supplied
    if vocab is None:
        aa_set = set( chain.from_iterable(tok_sequences) )
        sorted_aa = sorted(aa_set)
        vocab = {aa: ind for ind, aa in enumerate(sorted_aa, 1)}

    # 3. Encode
    vocab_size = max(vocab.values())
    if seq_len == 0:
        seq_length = max(map(len, tok_sequences))

    enc_sequences = np.zeros((len(tok_sequences), 
                              seq_length,
                              vocab_size),
                             dtype=np.float32)
    for row_ind, seq in enumerate(tok_sequences):
        for col_ind, tok in enumerate(seq):
            enc_sequences[row_ind, col_ind, vocab[tok] - 1] = 1.

    return enc_sequences

In [9]:
print(
    one_hot_encode(sequences)
    )

[[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0.

## **Bonus**: A scikit-learn compatable encoder

In many situations it may be more useful to have pre-made solution which can interface with common machine learning packages such as scikit-learn. I make use of this in my own work to build longer pipelines for data processing. Bellow I will give a brief demo of a tool that I put together specifically for this purpose, which is available as a [**gist**](https://gist.github.com/AnthonyOfSeattle/43b932bcb9b5b4b00ccbe96c29769db9). In subsequent notebooks, I will show how this can be used alongside other models with a scikit-learn based interface to make pipelines.

In [10]:
# The following is included to download the raw gist locally
import os
import requests

if not os.path.exists("sequence_encoder.py"): 
    url = "https://gist.githubusercontent.com/AnthonyOfSeattle/43b932bcb9b5b4b00ccbe96c29769db9/raw/a7a8bf8018a1e9a7b6900dfc61230718ea8843fb/sequence_encoder.py"
    r = requests.get(url)
    with open("sequence_encoder.py", "wb") as f:
        f.write(r.content)

#### **1. Initilization**

In [11]:
from sequence_encoder import SequenceEncoder

help(SequenceEncoder.__init__)

Help on function __init__ in module sequence_encoder:

__init__(self, pattern='\\w', vocab=None, seq_len=0, one_hot=False, oov_warn=True)
    Create an encoder class with optional fixed parameters.
    
    The most important parameter here is the regex `pattern`, which must be
    tailored to the given dataset. The optional `vocab` parameter can be
    used to pass a premade dictionary of token to integer correspondences.
    These will not be overwritten, but new tokens can still be learned.
    In the case that a predefined length is necessary, it can be determined
    by a possitive `seq_len` variable. Setting `seq_len` to a negative number,
    will allow the sequence lengths to by dyanmically determined. By default, 
    the class will return data with an integer encoding, but a one hot encoding 
    can be returned with `one_hot=True`. Finally, the class will spit out warnings 
    when a token is not found in the vocabulary. In these cases, the position will 
    be treated as 

Once loaded, the `SequenceEncoder` class can be initialized with the regex pattern of your choice, which will be used to parse the passed sequences. In some cases, it may be advantageous to define a global vocabulary, which can be passed with the vocab parameter. An object which perform in a similar fashion to the anaysis above can be created like so.

In [12]:
encoder = SequenceEncoder(pattern="[A-Z][^A-Z]*", seq_len=17, one_hot=True)

#### **2. Training and Transformation**

The real benefit of a tool like this is that it can be trained on a set of data, and maintain the mapping of amino acids to integers even on a new data set where some amino acids are missing. To train and store an amino acid to integer mapping, simply call the `fit` method.

In [13]:
sequences = ["IIRHPQYDR",
             "TLNNDIM[16]LIK",
             "VSTIS[80]LPTAPPATGTK"]

encoder.fit(sequences)

We can then transform the data using the `transform` function. A single sequence can be passed or a list of sequences.

In [14]:
print(
    encoder.transform("TLNNDIM[16]LIK")
)

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
